# LOADING DEPENDENCIES

## <span style="color:red"> MUST RUN ALWAYS</span>


In [1]:
# Data Manipulation
import pandas as pd
import numpy as np

# Checkpoints
import pickle
from tqdm.notebook import trange, tqdm

# Preprocessing
import re    # RegEx for removing non-letter characters
import nltk  #natural language processing
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import sklearn.preprocessing as pr


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/juliorenteria/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# LOADING DATASET

In [38]:
file_name = ''

data = pd.read_csv(filename)



In [40]:
data.head(50)

,text,sentiment
0,TRENDING: New Yorkers encounter empty supermar...,-1
1,When I couldn't find hand sanitizer at Fred Me...,1
2,Find out how you can protect yourself and love...,1
3,#Panic buying hits #NewYork City as anxious sh...,-1
4,#toiletpaper #dunnypaper #coronavirus #coronav...,0
5,Do you remember the last time you paid $2.99 a...,0
6,Voting in the age of #coronavirus = hand sanit...,1
7,"@DrTedros ""We cant stop #COVID19 without prot...",0
8,HI TWITTER! I am a pharmacist. I sell hand san...,-1
9,Anyone been in a supermarket over the last few...,1


# TOKENIZING

Separate tweets into lists of word (or components) that carry meanings.

Removing/converting the following elements:
- Uppercase
- Urls
- Simbols
- Numbers
- English Stopwords
- Stemming

In [ ]:
# Function to format, tokenize and remove stopwords from tweets.
def tweet_to_words(tweet):
    ''' Convert tweet text into a sequence of words '''
    
    # convert to lowercase
    text = tweet.lower()
    # remove tweeter users ( @xxx )
    text = re.sub(r"[@]\w+", " ", text)
    # remove https
    text = re.sub(r"[(http(s)?):\/\/(www\.)?a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&\/\/=]*)", " ", text)
    #text = " ".join(filter(lambda w: w.find("https://") == -1, text.split(" ")))
    # remove non letters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    #remove numbers
    text = re.sub(r"[0-9]*", "", text)
    # tokenize
    words = text.split()
    # remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    # apply stemming
    words = [PorterStemmer().stem(w) for w in words]
    # return list
    return words


In [ ]:
# Testing tweet_to_words Function
sel = 4
print("\nORIGINAL TWEET ->\n", data['text'][sel])
print("\nPROCESSED TWEET ->\n", tweet_to_words(data['text'][sel]))


In [ ]:
# Apply tokenization to each tweet and store as X
X = []
for i in trange(len(data['text'])):
    X.append(tweet_to_words(data['text'][i]))

In [ ]:
# Encode target labels
le = LabelEncoder()
Y = le.fit_transform(data['sentiment'])


In [ ]:
# Apply tokenization to each tweet and store as X
# X = list(map(tweet_to_words, data['text']))


## 🚩 CHECKPOINT 

Saving **Tokenized List** as a Pickle File to retrieve latter and save memory and time.

In [2]:
# Saving X to a PICKLE to retrieve back latter:

# Code to Save PICKLE
#with open('X.pkl', 'wb') as f:
#    pickle.dump(X, f)
#with open('Y.pkl', 'wb') as f:
#    pickle.dump(Y, f)


In [3]:
# Code to Retrieve PICKLE
#with open('X.pkl', 'rb') as f:
#    X = pickle.load(f)
#with open('Y.pkl', 'rb') as f:
#    Y = pickle.load(f)


# TRAIN TEST SPLIT

In [4]:
# Splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=13)

print('Number of tweets in the total set :    {}'.format(len(X)))
print('Number of tweets in the training set : {}'.format(len(X_train)))
print('Number of tweets in the testing set :  {}'.format(len(X_test)))


Number of tweets in the total set :    239252
Number of tweets in the training set : 167476
Number of tweets in the testing set :  71776


# BAG OF WORDS


In [5]:
# Convert a collection of text documents to a matrix of token counts
vocabulary_size = 5000    # Rounded up from 17422

# Generating Bag of Words
# Tweets have already been preprocessed hence dummy function will be passed in to preprocessor & tokenizer step
count_vector = CountVectorizer(max_features=vocabulary_size, preprocessor=lambda x: x, tokenizer=lambda x: x)
count_vector.fit(X_train)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=5000, min_df=1,
                ngram_range=(1, 1),
                preprocessor=<function <lambda> at 0x7f86f6c686a8>,
                stop_words=None, strip_accents=None,
                token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function <lambda> at 0x7f86f6c68598>,
                vocabulary=None)

In [10]:
# CountVectorizer creates a vocabulary. Checking Vocabulary
print('VOCABULARY SIZE: ', len(count_vector.vocabulary_))
print('VOCABULARY CONTENT: ', count_vector.vocabulary_)


VOCABULARY SIZE:  5000
VOCABULARY CONTENT:  {'modi': 2877, 'say': 3904, 'follow': 1741, 'think': 4496, 'fine': 1700, 'releas': 3701, 'movi': 2915, 'abus': 31, 'run': 3831, 'politician': 3374, 'elect': 1454, 'countri': 1046, 'cinema': 839, 'one': 3116, 'major': 2700, 'influenc': 2255, 'peopl': 3273, 'human': 2155, 'alway': 171, 'civil': 848, 'world': 4935, 'govern': 1916, 'putin': 3551, 'w': 4812, 'dream': 1392, 'rt': 3821, 'rise': 3786, 'fear': 1661, 'trigger': 4617, 'person': 3287, 'safeti': 3858, 'concern': 942, 'via': 4776, 'journey': 2403, 'common': 919, 'man': 2715, 'trailer': 4590, 'ashish': 286, 'sharma': 4002, 'congratul': 964, 'possibl': 3393, 'whether': 4887, 'black': 532, 'money': 2889, 'return': 3761, 'india': 2235, 'anybodi': 227, 'guess': 1961, 'white': 4891, 'leav': 2559, 'everybodi': 1545, 'see': 3945, 'nirav': 3035, 'mallya': 2709, 'carri': 723, 'full': 1814, 'report': 3728, 'card': 719, 'shock': 4030, 'chang': 775, 'last': 2533, 'year': 4968, 'suffer': 4324, 'highest'

In [7]:
# Transform the training data
X_train_bow = count_vector.transform(X_train).toarray()

# Transform the testing data
X_test_bow = count_vector.transform(X_test).toarray()


In [36]:
len(X_train_bow[1])

5000

## Experiments from here on out


In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier


In [28]:
Ran_For = RandomForestClassifier(max_depth=5, n_estimators=500, criterion='entropy')
Ran_For.fit(X_train_bow, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [37]:
Ran_For.predict(X_test_bow[0:500])

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [33]:

unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

{0: 43031, 1: 53394, 2: 71051}

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train_bow, y_train)

In [ ]:
neigh.score(X_test_bow, y_test)

In [ ]:
neigh.predict(X_test_bow[0:10])

In [ ]:
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)